In [1]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
import pandas_profiling as pp
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import json
import folium
import squarify
from datetime import date
import datetime
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import regex
import re
import random
from sklearn.utils import shuffle

In [2]:
df = pd.read_excel(r"C:\Users\nitishkumar\Desktop\P\Data-\Fullerton Msg\SMS_DATA\Raw sms\SMS Data_rawsms.xlsx",encoding='latin-1')
df

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId
0,110365150,076B042A-5AF4-47D3-A07D-36E20608A927,bce1f870eae211cf,Rs.12600 is Debited to A/c ...1545 on 09-08-18...,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,AXBOBTXN,314.0
1,110365151,076B042A-5AF4-47D3-A07D-36E20608A927,bce1f870eae211cf,Your VPA 7041699604@ybl linked to your a/c no....,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,AXBOBSMS,315.0
2,110365152,076B042A-5AF4-47D3-A07D-36E20608A927,bce1f870eae211cf,You have added Rs.200 to your PhonePe wallet f...,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807974427,TMPHONPE,316.0
3,110365153,076B042A-5AF4-47D3-A07D-36E20608A927,bce1f870eae211cf,Your a/c no. XXXXXXXX5232 is credited by Rs.20...,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808013405,BXSBIINB,317.0
4,110365154,076B042A-5AF4-47D3-A07D-36E20608A927,bce1f870eae211cf,Withdrawal of Rs.200 from your PhonePe wallet ...,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808019215,TMPHONPE,318.0
...,...,...,...,...,...,...,...,...,...
339328,110706373,354CED26-F98C-4E1A-A804-C2BDEC06479C,c84fd0f6561e019b,"Diabetes me kya khaye kya na khaye, Janiye Ayu...",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533355901258,VM-611121,1472.0
339329,110706374,354CED26-F98C-4E1A-A804-C2BDEC06479C,c84fd0f6561e019b,"Dear SBI Cardholder, Get upto Rs. 92000 at 1.1...",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533379603795,HP-611928,1475.0
339330,110706375,354CED26-F98C-4E1A-A804-C2BDEC06479C,c84fd0f6561e019b,Your Account xxxxxxxx001974xx has been CREDITE...,2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533388769632,AX-KTKBNK,1476.0
339331,110706376,354CED26-F98C-4E1A-A804-C2BDEC06479C,c84fd0f6561e019b,"x001974x DEBITED for Rs.10,000.00/- NFS-CWDR/+...",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533393068109,AX-KTKBNK,1477.0


In [3]:
df = df.apply(lambda x: x.astype(str).str.lower())

In [4]:
df['gUserId'].value_counts()

cfc552cc-b700-4b2a-b3e3-8e67c233c5da    4672
ac827fd1-3e01-46cb-a1e7-05ff3c9f10b0    3077
6d559358-fc2f-4a18-8fb3-f91403c4a825    2783
dcc6dcb9-10eb-438b-a1eb-3409032e6d4f    2368
94188554-8737-4aeb-abbf-416122ec13d9    2219
                                        ... 
db1129a2-e40d-42bd-8b53-de8cbaca7a7f       1
d1718910-6454-4b13-8607-67e46781131c       1
9e17d562-9793-413e-b5f2-34c4f6f3ed5b       1
66d7f7de-6835-4b50-b3f4-ca70c8fd74a7       1
6f86fc57-a319-48f8-b153-0b9c9d1f639e       1
Name: gUserId, Length: 2366, dtype: int64

In [5]:
pd.set_option('display.max_colwidth', -1)

## Fetch Amount from the SMS

## Debited

In [6]:
debited = df[df['message'].str.contains('debited').fillna(False)]

In [7]:
debited = debited[~debited.message.str.contains("tata sky")]

In [8]:
def split_it(amount):
    x = re.search('((([r][s])|([Ii][nN][rr]))\s{0,1}[.]{0,1}\s*((((\d{1,4}[,]{1})+\d{3})|\d+)[.]{0,1}\d{0,2}[/]{0,1}[-]{0,1}))', amount)
    if x :
      return(x.group())

In [9]:
debited['Debited Amount'] = debited['message'].apply(split_it)
debited

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Debited Amount
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,axbobtxn,314.0,rs.12600
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,axbobsms,315.0,rs. 200.0
5,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808303294,axbobtxn,319.0,rs.200
8,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533846772103,idcbssbi,326.0,inr 13.44
11,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876233446,adsbiupi,330.0,inr 86.0
...,...,...,...,...,...,...,...,...,...,...
339286,110706331,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"x001974x debited for rs.10,000.00/- nfs-cwdr/+bheem singh- balance is rs.6,502.63(06/07/2018:13:20:13)",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1530863729835,id-ktkbnk,1250.0,"rs.10,000.00/-"
339287,110706332,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"x001974x debited for rs.6,500.00/- nfs-cwdr/+bheem singh- balance is rs.2.63(06/07/2018:13:22:24)",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1530864550377,id-ktkbnk,1251.0,"rs.6,500.00/-"
339293,110706338,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,x001974x debited for rs.900.00/- nfs-cwdr/+tuglkabad e- balance is rs.4.06(12/07/2018:17:54:07),2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1531398377055,id-ktkbnk,1290.0,rs.900.00/-
339331,110706376,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"x001974x debited for rs.10,000.00/- nfs-cwdr/+bheem singh- balance is rs.196.06(04/08/2018:19:54:08)",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533393068109,ax-ktkbnk,1477.0,"rs.10,000.00/-"


In [10]:
debited['Debited Amount'].isna().sum()

0

In [11]:
debited['Category'] = 1

# df + debited

In [12]:
df_deb = pd.merge(df,debited,how = 'inner',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId'])

In [13]:
df_deb

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Debited Amount,Category
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,axbobtxn,314.0,rs.12600,1
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,axbobsms,315.0,rs. 200.0,1
2,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808303294,axbobtxn,319.0,rs.200,1
3,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533846772103,idcbssbi,326.0,inr 13.44,1
4,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876233446,adsbiupi,330.0,inr 86.0,1
...,...,...,...,...,...,...,...,...,...,...,...
43638,110706331,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"x001974x debited for rs.10,000.00/- nfs-cwdr/+bheem singh- balance is rs.6,502.63(06/07/2018:13:20:13)",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1530863729835,id-ktkbnk,1250.0,"rs.10,000.00/-",1
43639,110706332,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"x001974x debited for rs.6,500.00/- nfs-cwdr/+bheem singh- balance is rs.2.63(06/07/2018:13:22:24)",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1530864550377,id-ktkbnk,1251.0,"rs.6,500.00/-",1
43640,110706338,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,x001974x debited for rs.900.00/- nfs-cwdr/+tuglkabad e- balance is rs.4.06(12/07/2018:17:54:07),2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1531398377055,id-ktkbnk,1290.0,rs.900.00/-,1
43641,110706376,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"x001974x debited for rs.10,000.00/- nfs-cwdr/+bheem singh- balance is rs.196.06(04/08/2018:19:54:08)",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533393068109,ax-ktkbnk,1477.0,"rs.10,000.00/-",1


# Credited

In [14]:
credited = df[df['message'].str.contains('credited').fillna(False)]

In [15]:
credited = credited[~credited.message.str.contains("cashback")]

In [16]:
credited = credited[~credited.message.str.contains("vpa")]

In [17]:
credited['Credited Amount'] = credited['message'].apply(split_it)

In [18]:
credited['Credited Amount'].isna().sum()

0

In [19]:
credited['gUserId'].value_counts()

cfc552cc-b700-4b2a-b3e3-8e67c233c5da    506
7732a63b-e6f3-47d2-98be-c3a8b56170f5    381
4ec0af33-fc97-46af-8526-6078162a184c    377
15ad2a87-2c0b-4729-be66-73e1244744cd    325
ae1bcb2f-cf65-443c-8284-09046a9d23af    290
                                       ... 
3b714524-effa-4155-b4a9-6906899efa9a    1  
8df272d7-9033-4909-9f01-f971bfa983cb    1  
3f6082f9-bda7-4472-9762-74a383266818    1  
478320b3-b988-4d1b-95b0-c17e46f78449    1  
a25ebdee-ef43-491f-a62e-1b2405c994ef    1  
Name: gUserId, Length: 1620, dtype: int64

In [20]:
credited['Category'] = 2
credited

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Credited Amount,Category
3,110365153,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your a/c no. xxxxxxxx5232 is credited by rs.200.00 on 09-08-18 by a/c linked to mobile 8xxxxxx365-mohamadsahid salimbh (imps ref no 822115013298).,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808013405,bxsbiinb,317.0,rs.200.00,2
13,110365163,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"rs.186 is credited to a/c ...1545 on 10-08-18 10:15:01 (avlbl bal rs.198.6) at mumbai main office,mumbai. tollfree 1800223344 18001024455 (24x7)",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876424307,axbobtxn,332.0,rs.186,2
23,110365173,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"rs.100 is credited to a/c ...1545 on 10-08-18 18:08:36 (avlbl bal rs.298.6) at mumbai main office,mumbai. tollfree 1800223344 18001024455 (24x7)",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533904853905,axbobtxn,349.0,rs.100,2
24,110365174,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"rs.100 is credited to a/c ...1545 on 10-08-18 18:09:39 (avlbl bal rs.398.6) at mumbai main office,mumbai. tollfree 1800223344 18001024455 (24x7)",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533904866541,axbobtxn,350.0,rs.100,2
32,110365182,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your a/c no. xxxxxxxx5232 is credited by rs.100.00 on 11-08-18 by a/c linked to mobile 8xxxxxx365-mohamadsahid salimbh (imps ref no 822318018555). download yono www.yonosbi.com.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533993904691,idsbiinb,366.0,rs.100.00,2
...,...,...,...,...,...,...,...,...,...,...,...
339250,110706295,46cf036c-7f31-4a23-8e27-31a3953ac9e2,ff0bfccd767a001f,ac xxxxxxxx00043919 credited with rs.1500.00 13-08-2018 19:33:03 thru upi . aval bal rs.1526.35 cr. helpline 18001802222,2018-08-25 12:49:12.153000,2018-08-25 12:49:12.153000,1534168997682,hp-pnbsms,2686.0,rs.1500.00,2
339282,110706327,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,your account xxxxxxxx001974xx has been credited for rs.16692/neft-flying fabrication/8 clear balance is rs.16692.63-(2018-07-05 17:56:31)-contact@18004251444,2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1530794030070,ad-ktkbnk,1240.0,rs.16692/,2
339301,110706346,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"your a/c xxxxx850356 credited inr 10,000.00 on 16/07/18 -deposited by cash by d at gcc . a/c bal inr 11,151.30",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1531734207932,hp-cbssbi,1324.0,"inr 10,000.00",2
339304,110706349,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, we have credited your sbi atm payment of rs.10100 against your card xxxx2305. you can also use neft to pay your sbi card bill and get credit of funds in 1 working day. use ifsc code sbin00cards.",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1531971137918,hp-sbicrd,1347.0,rs.10100,2


# merge df_debited+ credited

In [21]:
df_deb_cred = pd.merge(df_deb,credited,how ='outer',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId','Category'])

In [22]:
df_deb_cred

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Debited Amount,Category,Credited Amount
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,axbobtxn,314.0,rs.12600,1,NaN
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,axbobsms,315.0,rs. 200.0,1,NaN
2,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808303294,axbobtxn,319.0,rs.200,1,NaN
3,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533846772103,idcbssbi,326.0,inr 13.44,1,NaN
4,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876233446,adsbiupi,330.0,inr 86.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
79100,110706295,46cf036c-7f31-4a23-8e27-31a3953ac9e2,ff0bfccd767a001f,ac xxxxxxxx00043919 credited with rs.1500.00 13-08-2018 19:33:03 thru upi . aval bal rs.1526.35 cr. helpline 18001802222,2018-08-25 12:49:12.153000,2018-08-25 12:49:12.153000,1534168997682,hp-pnbsms,2686.0,NaN,2,rs.1500.00
79101,110706327,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,your account xxxxxxxx001974xx has been credited for rs.16692/neft-flying fabrication/8 clear balance is rs.16692.63-(2018-07-05 17:56:31)-contact@18004251444,2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1530794030070,ad-ktkbnk,1240.0,NaN,2,rs.16692/
79102,110706346,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"your a/c xxxxx850356 credited inr 10,000.00 on 16/07/18 -deposited by cash by d at gcc . a/c bal inr 11,151.30",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1531734207932,hp-cbssbi,1324.0,NaN,2,"inr 10,000.00"
79103,110706349,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, we have credited your sbi atm payment of rs.10100 against your card xxxx2305. you can also use neft to pay your sbi card bill and get credit of funds in 1 working day. use ifsc code sbin00cards.",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1531971137918,hp-sbicrd,1347.0,NaN,2,rs.10100


In [23]:
df_deb_cred = df_deb_cred.drop_duplicates(subset = ['id'],keep='first')

In [24]:
df_deb_cred['id'].value_counts()

110675834    1
110569088    1
110511838    1
110505473    1
110368104    1
            ..
110601378    1
110444367    1
110573712    1
110488325    1
110698898    1
Name: id, Length: 71494, dtype: int64

## Credit Card Holders

In [25]:
credit_card= df[df.message.str.contains(pat = 'credit card[^0-9a-z]|credit limit[^0-9a-z]',regex = True)] 

In [26]:
credit_card['Credit_Card_Holder'] = 1
credit_card

C:\Users\nitishkumar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Credit_Card_Holder
7,110365157,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi cardholder,txn of rs.6,240.00 on card ending with xx6965 at one97 communications l has been declined due to insufficient credit limit on 09-08-18. your current available limit is rs.0.00. pls call us at 18601801290/39020202 for information on credit limit.",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533830185551,adsbicrd,323.0,1
15,110365165,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi cardholder,txn of rs.7,000.00 on card ending with xx6965 at one97 communications l has been declined due to insufficient credit limit on 10-08-18. your current available limit is rs.0.00. pls call us at 18601801290/39020202 for information on credit limit.",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533879471803,adsbicrd,334.0,1
19,110365169,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi cardholder,txn of rs.200.00 on card ending with xx6965 at sojitra petroleum has been declined due to insufficient credit limit on 10-08-18. your current available limit is rs.0.00. pls call us at 18601801290/39020202 for information on credit limit.",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533880646771,axsbicrd,338.0,1
20,110365170,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi cardholder,txn of rs.200.00 on card ending with xx6965 at sojitra petroleum has been declined due to insufficient credit limit on 10-08-18. your current available limit is rs.0.00. pls call us at 18601801290/39020202 for information on credit limit.",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533880677020,adsbicrd,339.0,1
49,110365199,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"rs.7,280.00 was spent on your sbi card ending 6965 at paytm on 13/08/18. available credit limit: rs.4,720.00.",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1534144174086,axsbicrd,394.0,1
...,...,...,...,...,...,...,...,...,...,...
339316,110706361,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, this holiday season enjoy encash of upto rs. 92000 & payback in emis as low as rs. 3145. this pre-approved amount is over and above your credit limit to book, sms fp to 56767. t&c",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1532836403324,hp-611928,1414.0,1
339318,110706363,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"book flexipay emis in just a few clicks with your sbi credit card. convert your trxn of rs 4635 done on 20-jul at anand swadeshi store, to easy monthly installment. book now sbicard.com/b/2qihhg1l1bz . t&c apply - sbi card",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533009547750,hp-611928,1432.0,1
339320,110706365,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, get upto rs. 92000 at 1.17 % p.m. for your financial needs with encash. this pre-approved amount is over and above your credit limit. to book, click https://www.sbicard.com/b/diici5230n9. t&c",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533032039163,hp-611405,1434.0,1
339323,110706368,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, this holiday season enjoy encash of upto rs. 92000 & payback in emis as low as rs. 3145. this pre-approved amount is over and above your credit limit to book, sms fp to 56767. t&c",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533181203849,hp-611928,1449.0,1


In [27]:
credit_card['gUserId'].value_counts()

5a661e76-a00e-46e6-a665-6ff0649de46d    799
6d559358-fc2f-4a18-8fb3-f91403c4a825    572
764ae44b-ac6f-4b4a-8567-48e30a1c2b37    377
16f79597-9e5c-4315-88fe-a0e953ca1d1f    299
85477f42-cbee-4d43-a057-06c4e864ae3b    295
                                       ... 
f8d2ca52-6d4e-4e28-b35b-261bc0e8529a    1  
7d15b8a9-ad2a-41bd-9474-c0d117a25847    1  
e0c10406-bb67-415f-9436-89cf9e8ecf0a    1  
335322a7-99cf-438e-b251-bedf6ba3504f    1  
05323db9-d851-40a5-b3f1-00df9f8052f4    1  
Name: gUserId, Length: 1050, dtype: int64

## Merging credit card with remaining

In [28]:
df_deb_cred_creditcardholder = pd.merge(df_deb_cred,credit_card,how ='outer',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId'])

In [29]:
df_deb_cred_creditcardholder

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Debited Amount,Category,Credited Amount,Credit_Card_Holder
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,axbobtxn,314.0,rs.12600,1.0,NaN,NaN
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,axbobsms,315.0,rs. 200.0,1.0,NaN,NaN
2,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808303294,axbobtxn,319.0,rs.200,1.0,NaN,NaN
3,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533846772103,idcbssbi,326.0,inr 13.44,1.0,NaN,NaN
4,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876233446,adsbiupi,330.0,inr 86.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91475,110706361,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, this holiday season enjoy encash of upto rs. 92000 & payback in emis as low as rs. 3145. this pre-approved amount is over and above your credit limit to book, sms fp to 56767. t&c",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1532836403324,hp-611928,1414.0,NaN,NaN,NaN,1.0
91476,110706363,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"book flexipay emis in just a few clicks with your sbi credit card. convert your trxn of rs 4635 done on 20-jul at anand swadeshi store, to easy monthly installment. book now sbicard.com/b/2qihhg1l1bz . t&c apply - sbi card",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533009547750,hp-611928,1432.0,NaN,NaN,NaN,1.0
91477,110706365,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, get upto rs. 92000 at 1.17 % p.m. for your financial needs with encash. this pre-approved amount is over and above your credit limit. to book, click https://www.sbicard.com/b/diici5230n9. t&c",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533032039163,hp-611405,1434.0,NaN,NaN,NaN,1.0
91478,110706368,354ced26-f98c-4e1a-a804-c2bdec06479c,c84fd0f6561e019b,"dear sbi cardholder, this holiday season enjoy encash of upto rs. 92000 & payback in emis as low as rs. 3145. this pre-approved amount is over and above your credit limit to book, sms fp to 56767. t&c",2018-08-25 12:49:13.920000,2018-08-25 12:49:13.920000,1533181203849,hp-611928,1449.0,NaN,NaN,NaN,1.0


In [30]:
df_deb_cred_creditcardholder['id'].value_counts()

110675834    1
110388726    1
110647344    1
110412095    1
110436320    1
            ..
110497479    1
110552317    1
110568428    1
110438180    1
110581195    1
Name: id, Length: 91480, dtype: int64

# Loan

In [31]:
loan= df[df.message.str.contains(pat = 'loan[^0-9a-z]|emi[^0-9a-z]',regex = True)] 

In [32]:
loan = loan[~loan.message.str.contains("agent")]
loan = loan[~loan.message.str.contains("low balance")]
loan = loan[~loan.message.str.contains("e-voucher")]
loan = loan[~loan.message.str.contains("talktime")]
loan = loan[~loan.message.str.contains("eligible")]
loan = loan[~loan.message.str.contains("offer")]
loan = loan[~loan.message.str.contains("instant")]
loan = loan[~loan.message.str.contains("%")]
loan = loan[~loan.message.str.contains("cashback")]
loan = loan[~loan.message.str.contains("e-gv")]
loan = loan[~loan.message.str.contains("kissht app")]
loan = loan[~loan.message.str.contains("win a free")]
loan = loan[~loan.message.str.contains("apply for")]
loan = loan[~loan.message.str.contains("pre-approved")]

In [33]:
loan['EMI_amount'] = loan['message'].apply(split_it)

loan = shuffle(loan)

In [34]:
loan['ongoing_loan'] = 1
loan['Category'] = 3

In [35]:
loan['gUserId'].value_counts()

dcc6dcb9-10eb-438b-a1eb-3409032e6d4f    159
cfc552cc-b700-4b2a-b3e3-8e67c233c5da    148
00d2aece-c141-4c6a-8b7d-2a49fd2382f5    144
aa65dfec-67ca-4e20-a9aa-54eb9d926329    122
51505a75-29a7-4502-9078-eeca7fd02b68    121
                                       ... 
babfecf6-7547-4c9c-ba88-eb41b59e014a    1  
a2389607-dbd8-4113-8eba-e367ce42f2ab    1  
83e93864-b689-47c9-a4ad-3c5946f27f82    1  
52aadfa6-eecc-40a9-869a-3bfcdab69382    1  
de43279c-2583-46cd-8e5e-b33ae334971d    1  
Name: gUserId, Length: 1117, dtype: int64

In [36]:
loan

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,EMI_amount,ongoing_loan,Category
247,110365397,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,"gold loan: mul-195, interest due up to date rs.6740/-. pay online at https://online.muthootfinance.com. call 033-24074345. e&oe.",2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1525914952917,tm-muthut,3194.0,rs.6740/-,1,3
248,110365398,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,"gold loan: mul-253, interest due up to date rs.6102/-. pay online at https://online.muthootfinance.com. call 033-24074345. e&oe.",2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1526087772648,tm-muthut,3228.0,rs.6102/-,1,3
249,110365399,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,"gold loan: mul-465, interest due up to date rs.2652/-. pay online at https://online.muthootfinance.com. call 033-24074345. e&oe.",2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1526692544304,tm-muthut,3473.0,rs.2652/-,1,3
250,110365400,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,"gold loan: mul-195, interest due up to date rs.7647/-. pay online at https://online.muthootfinance.com. call 033-24074345. e&oe.",2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1528506936551,ad-muthut,3883.0,rs.7647/-,1,3
251,110365401,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,"gold loan: mul-253, interest due up to date rs.6922/-. pay online at https://online.muthootfinance.com. call 033-24074345. e&oe.",2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1528679843545,ad-muthut,3915.0,rs.6922/-,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
339038,110706083,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"greetings! emi of rs.1,985/- for loan 4050pl21073619 is due on 05-aug-18. keep your icici bank a/c -4312 funded to avoid bounce penal and overdue charges.for queries, chat with our virtual assistant blu on http://m.bajfin.in/eic5jegu . regards,bajaj finserv",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1533282693668,im-bajajf,527.0,"rs.1,985/-",1,3
339083,110706128,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00 towards fees and charges.you can make payment at https://goo.gl/yvuayf making this payment will help you improve your credit scores and enrich your financial profile.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534421101147,ad-hdfcbk,674.0,"rs.10,737.00",1,3
339101,110706146,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00 towards fees and charges.you can make payment at https://goo.gl/yvuayf making this payment will help you improve your credit scores and enrich your financial profile.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534848428287,ad-hdfcbk,714.0,"rs.10,737.00",1,3
339252,110706297,46cf036c-7f31-4a23-8e27-31a3953ac9e2,ff0bfccd767a001f,"dear customer,you now have over 2 emi payments due for your hdfc bank -consumer durable life style product-xx0058 missing payments are indicators used to identify potential defaulters.please make a payment of rs.2679.80.and revert to non-defaulting status",2018-08-25 12:49:12.153000,2018-08-25 12:49:12.153000,1534521550038,mdhdfccc,2923.0,rs.2679.80,1,3


## merging remaining with loan 

In [37]:
df_deb_cred_creditcardholder_loan = pd.merge(df_deb_cred_creditcardholder,loan,how ='outer',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId','Category'])

In [38]:
df_deb_cred_creditcardholder_loan = df_deb_cred_creditcardholder_loan.drop_duplicates(subset = ['id'],keep='last')

In [39]:
df_deb_cred_creditcardholder_loan

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Debited Amount,Category,Credited Amount,Credit_Card_Holder,EMI_amount,ongoing_loan
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,axbobtxn,314.0,rs.12600,1.0,NaN,NaN,NaN,NaN
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,axbobsms,315.0,rs. 200.0,1.0,NaN,NaN,NaN,NaN
2,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808303294,axbobtxn,319.0,rs.200,1.0,NaN,NaN,NaN,NaN
3,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533846772103,idcbssbi,326.0,inr 13.44,1.0,NaN,NaN,NaN,NaN
4,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876233446,adsbiupi,330.0,inr 86.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104962,110706083,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"greetings! emi of rs.1,985/- for loan 4050pl21073619 is due on 05-aug-18. keep your icici bank a/c -4312 funded to avoid bounce penal and overdue charges.for queries, chat with our virtual assistant blu on http://m.bajfin.in/eic5jegu . regards,bajaj finserv",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1533282693668,im-bajajf,527.0,NaN,3.0,NaN,NaN,"rs.1,985/-",1.0
104963,110706128,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00 towards fees and charges.you can make payment at https://goo.gl/yvuayf making this payment will help you improve your credit scores and enrich your financial profile.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534421101147,ad-hdfcbk,674.0,NaN,3.0,NaN,NaN,"rs.10,737.00",1.0
104964,110706146,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00 towards fees and charges.you can make payment at https://goo.gl/yvuayf making this payment will help you improve your credit scores and enrich your financial profile.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534848428287,ad-hdfcbk,714.0,NaN,3.0,NaN,NaN,"rs.10,737.00",1.0
104965,110706297,46cf036c-7f31-4a23-8e27-31a3953ac9e2,ff0bfccd767a001f,"dear customer,you now have over 2 emi payments due for your hdfc bank -consumer durable life style product-xx0058 missing payments are indicators used to identify potential defaulters.please make a payment of rs.2679.80.and revert to non-defaulting status",2018-08-25 12:49:12.153000,2018-08-25 12:49:12.153000,1534521550038,mdhdfccc,2923.0,NaN,3.0,NaN,NaN,rs.2679.80,1.0


In [40]:
df_deb_cred_creditcardholder_loan['id'].value_counts()

110675834    1
110460753    1
110464543    1
110599811    1
110545221    1
            ..
110411903    1
110450592    1
110376728    1
110653812    1
110669007    1
Name: id, Length: 103637, dtype: int64

df_deb_cred_creditcardholder_loan[df_deb_cred_creditcardholder_loan['id'].str.contains('110582888').fillna(False)]

# Defaulters

In [41]:
Defaulters= loan[loan.message.str.contains(pat = 'pending[^0-9a-z]|overdue[^0-9a-z]|pls pay[^0-9a-z]',regex = True)] 

In [42]:
Defaulters['Overdue_amount'] = Defaulters['message'].apply(split_it)

C:\Users\nitishkumar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
Defaulters['User_is_defaulter'] = 1
Defaulters

C:\Users\nitishkumar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,EMI_amount,ongoing_loan,Category,Overdue_amount,User_is_defaulter
990,110366140,bf6110ec-68fa-4c74-b560-9890686713fb,b3486d04812a86b8,"the instalment for your l&t two wheeler loan no. 3095218¿still remains overdue¿despite several reminders. please pay the overdue amt of rs. 2284 along with bouncing & lpc charges. please ignore if paid. for any queries , contact 18002094747 l&t finance ltd.",2018-08-25 07:43:27.270000,2018-08-25 07:43:27.270000,1535026857251,tx-lntfin,6794.0,rs. 2284,1,3,rs. 2284,1
1014,110366164,bf6110ec-68fa-4c74-b560-9890686713fb,b3486d04812a86b8,"the instalment for your l&t two wheeler loan no. 3095218æstill remains overdueædespite several reminders. please pay the overdue amt of rs. 2284 along with bouncing & lpc charges. please ignore if paid. for any queries , contact 18002094747 l&t finance ltd.",2018-08-25 07:43:27.270000,2018-08-25 07:43:27.270000,1534943211040,vk-lntfin,6782.0,rs. 2284,1,3,rs. 2284,1
1371,110366521,06913263-cdf0-420c-80b8-21712d98b604,ebc57dc8470b9a78,"greetings! emi of rs.2,081/- for loan 400cdd79812660 is due on 02-aug-18. keep your sbi bank a/c -0922 funded to avoid bounce penal and overdue charges.for queries, chat with our virtual assistant blu on http://m.bajfin.in/jnpmloy0 . regards,bajaj finserv",2018-08-25 07:44:10.583000,2018-08-25 07:44:10.583000,1532962726495,ad-bajajf,410.0,"rs.2,081/-",1,3,"rs.2,081/-",1
1963,110367113,bcd97bf7-4d1b-4662-9f09-e783de23d370,40bdf454feb3910c,"dear client, your gold loan for rs. 21000/-is approved. disb. code 74718662. pls pay monthly interest/renew online thru convenient mobile app. http://msg.mn/9exqvm/a3zpmab.",2018-08-25 07:45:48.853000,2018-08-25 07:45:48.853000,1522137014245,tx-mafilt,54.0,rs. 21000/-,1,3,rs. 21000/-,1
1998,110367148,bcd97bf7-4d1b-4662-9f09-e783de23d370,40bdf454feb3910c,"dear customer,your loan a/c 2172***1500002 with iob is overdue for 10 days by rs.3,819.80 as on 07-apr-18 [sma-0].please regularize.",2018-08-25 07:45:48.853000,2018-08-25 07:45:48.853000,1523412099739,tm-iobchn,175.0,"rs.3,819.80",1,3,"rs.3,819.80",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338992,110706037,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer,a gentle reminder to remit overdue payment of rs.12,827.00 towards your hdfc bank -personal loan-51719303.timely payment will help you avoid levy of fees and charges.please log onto https://goo.gl/yvuayf to remit this amount.kindly ignore if you have already paid",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1532430547087,ad-hdfcbk,377.0,"rs.12,827.00",1,3,"rs.12,827.00",1
339038,110706083,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"greetings! emi of rs.1,985/- for loan 4050pl21073619 is due on 05-aug-18. keep your icici bank a/c -4312 funded to avoid bounce penal and overdue charges.for queries, chat with our virtual assistant blu on http://m.bajfin.in/eic5jegu . regards,bajaj finserv",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1533282693668,im-bajajf,527.0,"rs.1,985/-",1,3,"rs.1,985/-",1
339083,110706128,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00 towards fees and charges.you can make payment at https://goo.gl/yvuayf making this payment will help you improve your credit scores and enrich your financial profile.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534421101147,ad-hdfcbk,674.0,"rs.10,737.00",1,3,"rs.10,737.00",1
339101,110706146,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00

## merge remaining with defaulters

In [44]:
df_defaulters = pd.merge(df_deb_cred_creditcardholder_loan,Defaulters,how ='outer',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId','Category','EMI_amount','ongoing_loan'])

In [45]:
df_defaulters

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Debited Amount,Category,Credited Amount,Credit_Card_Holder,EMI_amount,ongoing_loan,Overdue_amount,User_is_defaulter
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,axbobtxn,314.0,rs.12600,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,axbobsms,315.0,rs. 200.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808303294,axbobtxn,319.0,rs.200,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533846772103,idcbssbi,326.0,inr 13.44,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876233446,adsbiupi,330.0,inr 86.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103632,110706083,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"greetings! emi of rs.1,985/- for loan 4050pl21073619 is due on 05-aug-18. keep your icici bank a/c -4312 funded to avoid bounce penal and overdue charges.for queries, chat with our virtual assistant blu on http://m.bajfin.in/eic5jegu . regards,bajaj finserv",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1533282693668,im-bajajf,527.0,NaN,3.0,NaN,NaN,"rs.1,985/-",1.0,"rs.1,985/-",1.0
103633,110706128,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00 towards fees and charges.you can make payment at https://goo.gl/yvuayf making this payment will help you improve your credit scores and enrich your financial profile.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534421101147,ad-hdfcbk,674.0,NaN,3.0,NaN,NaN,"rs.10,737.00",1.0,"rs.10,737.00",1.0
103634,110706146,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer.thanks for your recent payment.your hdfc bank personal loan 51719303 is now in much better health.to fully regularize your account, remit a further payment of rs.10,737.00 towards pending emi and rs.2,283.00 towards fees and charges.you can make payment at https://goo.gl/yvuayf making this payment will help you improve your credit scores and enrich your financial profile.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534848428287,ad-hdfcbk,714.0,NaN,3.0,NaN,NaN,"rs.10,737.00",1.0,"rs.10,737.00",1.0
103635,110706297,46cf036c-7f31-4a23-8e27-31a3953ac9e2,ff0bfccd767a001f,"dear customer,you now have over 2 emi payments due for your hdfc bank -consumer durable life style product-xx0058 missing payments are indicators used to identify potential defaulters.please make a payment of rs.2679.80.and revert to non-defaulting status",2018-08-25 12:49:12.153000,2018-08-25 12:49:12.153000,1534521550038,mdhdfccc,2923.0,NaN,3.0,NaN,NaN,rs.2679.80,1.0,NaN,NaN


In [46]:
df_defaulters['gUserId'].value_counts()

cfc552cc-b700-4b2a-b3e3-8e67c233c5da    1333
7732a63b-e6f3-47d2-98be-c3a8b56170f5    998 
5a661e76-a00e-46e6-a665-6ff0649de46d    907 
6d559358-fc2f-4a18-8fb3-f91403c4a825    845 
764ae44b-ac6f-4b4a-8567-48e30a1c2b37    827 
                                       ...  
d19d7bfb-180c-4631-95d1-f0bcab4f7608    1   
aba3c88e-642b-426b-8e20-7294bd0048b5    1   
b6884f05-a4b7-4c96-b8d6-56fb774f2918    1   
ba21777f-59c4-4516-be27-4f6370b280fb    1   
4734c462-4c22-4182-8f7c-82e67f433b52    1   
Name: gUserId, Length: 1947, dtype: int64

## Paytm Wallet

In [47]:
paytm= df[df.message.str.contains(pat = 'paytm[^0-9a-z]',regex = True)] 

In [48]:
paytm = paytm[~paytm.message.str.contains("oyo")]
paytm = paytm[~paytm.message.str.contains("don't worry")]
paytm = paytm[~paytm.message.str.contains("gb/day")]
paytm = paytm[~paytm.message.str.contains("mychoize")]

In [49]:
paytm['gUserId'].value_counts()

cfc552cc-b700-4b2a-b3e3-8e67c233c5da    504
02f0b2be-2926-48c3-9d97-587af3fb71e2    290
764ae44b-ac6f-4b4a-8567-48e30a1c2b37    274
c3ebfb71-fa4e-41c9-bb59-4735c68c7790    255
f2ac1876-d64e-4467-9206-295c4a4f4a3a    229
                                       ... 
ffe09d94-24d5-4c6e-a4fe-24cca92a89de    1  
b7b88a2c-221a-4d3f-9e10-6b3f99b0ff63    1  
3a95fe96-0493-40c6-9b4c-5ab1c9634174    1  
c1bf2ef5-e0dc-4864-942d-c40f8be1288b    1  
6d355511-e09f-440b-b54d-d45f4a39f57f    1  
Name: gUserId, Length: 1023, dtype: int64

In [50]:
paytm['paytm_user'] = 1
paytm

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,paytm_user
47,110365197,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,360634 is the otp for trxn of inr 7280.00 at paytm with your sbi card ending 6965. otp is valid for 10 mins only. pls do not share with anyone.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1534144152158,axsbicrd,392.0,1
48,110365198,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,360634 is the otp for trxn of inr 7280.00 at paytm with your sbi card ending 6965. otp is valid for 10 mins only. pls do not share with anyone.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1534144159466,axsbicrd,393.0,1
49,110365199,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"rs.7,280.00 was spent on your sbi card ending 6965 at paytm on 13/08/18. available credit limit: rs.4,720.00.",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1534144174086,axsbicrd,394.0,1
65,110365215,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"book flexipay emis in just a few clicks with your sbi credit card. convert your trxn of rs 7280 done on 13-aug at paytm, to easy monthly installment. book now sbicard.com/b/0nigqg1n0yi . t&c apply - sbi card",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1534396445878,ax040000,434.0,1
69,110365219,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"book flexipay emis in just a few clicks with your sbi credit card. convert your trxn of rs 7280 done on 13-aug at paytm, to easy monthly installment. book now sbicard.com/b/0nigqg1n0yi . t&c apply - sbi card",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1534649719930,ax040000,468.0,1
...,...,...,...,...,...,...,...,...,...,...
338995,110706040,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,delight with a flight! gift a moment of a lifetime with indigo gift vouchers. get 5% cashback upto inr100 on transactions through paytm https://bit.ly/2cicego,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1532495930947,ax-indoff,382.0,1
339022,110706067,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer,txn of inr 200.00 done via debit card of acct xx4312 on 31-jul-18.info: iin*paytm .avbl bal:inr 25,621.46.for dispute,call 04033667777.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1533047703118,ix-icicib,496.0,1
339024,110706069,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"cancelled: the seller has cancelled the grab any burger of your choice free (both web & app) from your paytm order. we have successfully initiated a refund of rs.0.get rs 200 cashback in july, use code: onceamonth t&c apply http://m.p-y.tm/oncm",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1533048507827,ax-ipaytm,498.0,1
339072,110706117,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer, acct xx4312 is credited with rs.14400.00 on 13-aug-18 from 8492801001@paytm. upi ref no 822515862196",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1534154301384,ad-icicib,647.0,1


## Paytm Debit

In [51]:
paytm_debit= paytm[paytm.message.str.contains(pat = 'paid[^0-9a-z]|withdrawn using your paytm rupay debit card[^0-9a-z]|transferred[^0-9a-z]|txn[^0-9a-z]',regex = True)] 

In [52]:
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("recharge your d2h")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("updated gold balance")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("paytm wallet dwara recharge karein aur payein")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("body checkup")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("muft")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("you have just unlocked")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("your emi for loan a/c")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("has added")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("received")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("loan")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("ipl")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("hurry")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("ab paye")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("1mg")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("mticket")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("get rs 200 cashback on shopping of rs")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("myidea")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("welcome gift")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("monthly groceries")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("get rs")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("http://m.p-y.tm/ltc")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("vespa & aprilia")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("welcome to paytm payments bank")]
paytm_debit = paytm_debit[~paytm_debit.message.str.contains("refunded")]

paytm_debit = shuffle(paytm_debit)

In [53]:
paytm_debit['Paytm_debit_trans_amount'] = paytm_debit['message'].apply(split_it)

In [54]:
paytm_debit

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,paytm_user,Paytm_debit_trans_amount
305,110365455,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,you have initiated a txn of inr 4100.00 at paytm on icici bank card 9009. otp is 534294. dont share it with anyone. bank never calls to verify otp.,2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1533872049741,tm-icicib,4890.0,1,inr 4100.00
307,110365457,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,you have initiated a txn of inr 3900.00 at paytm on icici bank card 9009. otp is 990911. dont share it with anyone. bank never calls to verify otp.,2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1533872117149,tm-icicib,4892.0,1,inr 3900.00
309,110365459,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,you have initiated a txn of inr 3600.00 at paytm on icici bank card 9009. otp is 240247. dont share it with anyone. bank never calls to verify otp.,2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1533872316236,tm-icicib,4894.0,1,inr 3600.00
311,110365461,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,you have initiated a txn of inr 3400.00 at paytm on icici bank card 9009. otp is 373348. dont share it with anyone. bank never calls to verify otp.,2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1533872463959,tm-icicib,4896.0,1,inr 3400.00
313,110365463,a570a1a4-6f66-4f51-98fd-5eba684eaa99,23e7547b39a306d1,you have initiated a txn of inr 2920.00 at paytm on icici bank card 9009. otp is 307663. dont share it with anyone. bank never calls to verify otp.,2018-08-25 07:42:22.410000,2018-08-25 07:42:22.410000,1533872554896,tm-icicib,4898.0,1,inr 2920.00
...,...,...,...,...,...,...,...,...,...,...,...
338223,110705268,869939e2-7cbf-4da9-932b-70b9e7a52071,f5c3010d86283b94,otp is 601394 for txn of inr 220.00 at paytm on card ending 6506. valid till 18:00:27. do not share otp for security reasons.,2018-08-25 12:46:35.977000,2018-08-25 12:46:35.977000,1533385654440,vk-hdfcbk,4498.0,1,inr 220.00
338524,110705569,41d52045-1ba1-4aaf-b6f9-40d747d68c06,c0ee14cbbac6ddd2,you have initiated a txn of inr 1.00 at paytm on icici bank card no.3470. otp is 135930. dont share with anyone. bank never calls to verify otp.,2018-08-25 12:47:01.350000,2018-08-25 12:47:01.350000,1534406858613,ad-icicib,4086.0,1,inr 1.00
338933,110705978,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,you have initiated a txn of inr 696.00 at paytm on icici bank card no.0803. otp is 693391. dont share with anyone. bank never calls to verify otp.,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1530627485461,tm-icicib,157.0,1,inr 696.00
338934,110705979,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer,txn of inr 696.00 done via debit card of acct xx4312 on 03-jul-18.info: iin*paytm .avbl bal:inr 5,409.26.for dispute,call 04033667777.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1530627493139,ix-icicib,158.0,1,inr 696.00


In [55]:
paytm_debit['gUserId'].value_counts()

02f0b2be-2926-48c3-9d97-587af3fb71e2    140
cfc552cc-b700-4b2a-b3e3-8e67c233c5da    140
b0818a6c-1cb7-45b9-83b6-9064ec4d2d0d    127
f2ac1876-d64e-4467-9206-295c4a4f4a3a    108
ec0ed6d5-e4d2-4c07-9859-86f6f09ecf38    92 
                                        .. 
925d94bf-010e-425a-8dd5-a5ba04b8569f    1  
df93027a-1051-4dff-b2a4-6efc511d94e3    1  
ad396748-aa03-43f2-8cab-0f3dd0fbe0eb    1  
c646a87a-0804-4c46-9789-02f79e4a24b7    1  
6bdcc47d-4ec3-4c24-afcd-cdfdd76c5454    1  
Name: gUserId, Length: 450, dtype: int64

In [56]:
paytm_debit['id'].value_counts()

110655039    1
110454417    1
110554238    1
110493777    1
110556267    1
            ..
110368224    1
110552703    1
110544911    1
110367029    1
110635132    1
Name: id, Length: 4181, dtype: int64

## Paytm Credit

In [57]:
paytm_credit= paytm[paytm.message.str.contains(pat = 'received[^0-9a-z]|we have added[^0-9a-z]|credited in[^0-9a-z]',regex = True)] 

paytm_credit = shuffle(paytm_credit)

In [58]:
paytm_credit['gUserId'].value_counts()

cfc552cc-b700-4b2a-b3e3-8e67c233c5da    157
a350748e-44db-46f6-a3cf-18ae2f3f6bd0    56 
dfd6fc55-da00-449a-a7e5-fcf13920c906    41 
c3ebfb71-fa4e-41c9-bb59-4735c68c7790    36 
f2ac1876-d64e-4467-9206-295c4a4f4a3a    30 
                                        .. 
72310dee-f4f4-40b4-8d99-6ec481bf6497    1  
8ed40073-66f5-41b2-a769-74b0bbc1a8f3    1  
16bbd301-6653-47c9-9946-574fb6d4900b    1  
03a656d5-00b2-4510-b624-d8c9576a3fe0    1  
0588d213-b017-49be-9ea2-bfc3f86ecb37    1  
Name: gUserId, Length: 199, dtype: int64

In [59]:
paytm_credit = paytm_credit[~paytm_credit.message.str.contains("request")]

In [60]:
paytm_credit['Paytm_credit_trans_amount'] = paytm_credit['message'].apply(split_it)
paytm_credit

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,paytm_user,Paytm_credit_trans_amount
431,110365581,243f8552-9132-42b5-b60f-fd131fa4e930,76a540ebc795689d,"dear customer, you have received rs.10000.00 in your account xxxxxxxx7972 from irfan ghouse via paytm wallet mobile xxxxxx7972. ref no: 817600180299. queries? write to us at http://m.p-y.tm/care",2018-08-25 07:42:34.673000,2018-08-25 07:42:34.673000,1529865023313,adpaytmb,215.0,1,rs.10000.00
462,110365612,243f8552-9132-42b5-b60f-fd131fa4e930,76a540ebc795689d,"dear customer, you have received rs.10000.00 in your account xxxxxxxx7972 from irfan ghouse via paytm wallet mobile xxxxxx7972. ref no: 818417182192. queries? write to us at http://m.p-y.tm/care",2018-08-25 07:42:34.673000,2018-08-25 07:42:34.673000,1530620226481,axpaytmb,386.0,1,rs.10000.00
581,110365731,243f8552-9132-42b5-b60f-fd131fa4e930,76a540ebc795689d,"dear customer, you have received rs.100.00 in your account xxxxxxxx7972 from irfan ghouse via paytm wallet mobile xxxxxx7972. ref no: 817019263469. queries? write to us at http://m.p-y.tm/care",2018-08-25 07:42:34.673000,2018-08-25 07:42:34.673000,1529415602905,vk-paytmb,75.0,1,rs.100.00
582,110365732,243f8552-9132-42b5-b60f-fd131fa4e930,76a540ebc795689d,"dear customer, you have received rs.9000.00 in your account xxxxxxxx7972 from irfan ghouse via paytm wallet mobile xxxxxx7972. ref no: 817019273044. queries? write to us at http://m.p-y.tm/care",2018-08-25 07:42:34.673000,2018-08-25 07:42:34.673000,1529416476096,vk-paytmb,77.0,1,rs.9000.00
583,110365733,243f8552-9132-42b5-b60f-fd131fa4e930,76a540ebc795689d,"dear customer, you have received rs.4000.00 in your account xxxxxxxx7972 from irfan ghouse via paytm wallet mobile xxxxxx7972. ref no: 817019273963. queries? write to us at http://m.p-y.tm/care",2018-08-25 07:42:34.673000,2018-08-25 07:42:34.673000,1529416578861,vk-paytmb,78.0,1,rs.4000.00
...,...,...,...,...,...,...,...,...,...,...,...
332860,110699799,39f07f36-12c6-4b46-8da2-423212b74a56,e1fba8438b2999fb,"dear customer, you have received rs.1000.00 in your savings account xxxxxxxx3330 from krunal lalitbhai chandvaniya c via paytm wallet mobile xxxxxx3330. ref no: 807812371917. queries? write to us at http://m.p-y.tm/care",2018-08-25 12:41:58.497000,2018-08-25 12:41:58.497000,1521443372049,vm-paytmb,4273.0,1,rs.1000.00
335700,110702744,60449b33-8cb9-4ab0-bf79-1c32ebc283a1,bc9ca9b3069d1fb7,received rs.250 from seva ravi (97xxxx5224) in your paytm wallet. wallet txn id: 18673610079. 100% cashback on first shopping order http://m.p-y.tm/firt .,2018-08-25 12:43:01.373000,2018-08-25 12:43:01.373000,1524902709560,ad-ipaytm,1621.0,1,rs.250
335776,110702820,60449b33-8cb9-4ab0-bf79-1c32ebc283a1,bc9ca9b3069d1fb7,received rs.225 from lakhyajit hazarika (73xxxx4219) in your paytm wallet. wallet txn id: 18866336137. get some exciting gifts for your mother http://m.p-y.tm/mdy .,2018-08-25 12:43:01.373000,2018-08-25 12:43:01.373000,1526197487767,ax-ipaytm,1893.0,1,rs.225
336683,110703728,4a8f8dac-e930-44f2-b297-88c15ee51d45,a6f2a2a12aa4747a,received rs.3200 from anant pratap singh (87xxxx2918) in your paytm wallet. wallet txn id: 18931888136. up to rs 150 cashback at grocery stores http://m.p-y.tm/jt91 .,2018-08-25 12:45:26.800000,2018-08-25 12:45:26.800000,1526641676289,ax-ipaytm,104.0,1,rs.3200


In [61]:
paytm_credit['id'].value_counts()

110555427    1
110573253    1
110519669    1
110634237    1
110554867    1
            ..
110429855    1
110496721    1
110663232    1
110595101    1
110655595    1
Name: id, Length: 1157, dtype: int64

# Merge paytm tables 

In [63]:
df_paytm = pd.merge(paytm,paytm_debit,how = 'outer',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId','paytm_user'])

In [65]:
df_paytm_final = pd.merge(df_paytm,paytm_credit,how = 'outer',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId','paytm_user'])

In [66]:
Final =  pd.merge(df_defaulters,df_paytm_final,how = 'outer',on = ['id','gUserId','deviceId','message','createdDate','modifiedDate','smsDate','sender_name','messageId'])

In [70]:
Final['id'].value_counts()

110675834    1
110627535    1
110612916    1
110664798    1
110529983    1
            ..
110532186    1
110695896    1
110598049    1
110655201    1
110669007    1
Name: id, Length: 116800, dtype: int64

In [73]:
Final = Final.fillna(0)

In [74]:
Final

,id,gUserId,deviceId,message,createdDate,modifiedDate,smsDate,sender_name,messageId,Debited Amount,Category,Credited Amount,Credit_Card_Holder,EMI_amount,ongoing_loan,Overdue_amount,User_is_defaulter,paytm_user,Paytm_debit_trans_amount,Paytm_credit_trans_amount
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533799814965,axbobtxn,314.0,rs.12600,1.0,0,0.0,0,0.0,0,0.0,0.0,0,0
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533807964427,axbobsms,315.0,rs. 200.0,1.0,0,0.0,0,0.0,0,0.0,0.0,0,0
2,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533808303294,axbobtxn,319.0,rs.200,1.0,0,0.0,0,0.0,0,0.0,0.0,0,0
3,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533846772103,idcbssbi,326.0,inr 13.44,1.0,0,0.0,0,0.0,0,0.0,0.0,0,0
4,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,1533876233446,adsbiupi,330.0,inr 86.0,1.0,0,0.0,0,0.0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116795,110705978,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,you have initiated a txn of inr 696.00 at paytm on icici bank card no.0803. otp is 693391. dont share with anyone. bank never calls to verify otp.,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1530627485461,tm-icicib,157.0,0,0.0,0,0.0,0,0.0,0,0.0,1.0,inr 696.00,0
116796,110705979,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer,txn of inr 696.00 done via debit card of acct xx4312 on 03-jul-18.info: iin*paytm .avbl bal:inr 5,409.26.for dispute,call 04033667777.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1530627493139,ix-icicib,158.0,0,0.0,0,0.0,0,0.0,0,0.0,1.0,inr 696.00,0
116797,110706040,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,delight with a flight! gift a moment of a lifetime with indigo gift vouchers. get 5% cashback upto inr100 on transactions through paytm https://bit.ly/2cicego,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1532495930947,ax-indoff,382.0,0,0.0,0,0.0,0,0.0,0,0.0,1.0,0,0
116798,110706067,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,"dear customer,txn of inr 200.00 done via debit card of acct xx4312 on 31-jul-18.info: iin*paytm .avbl bal:inr 25,621.46.for dispute,call 04033667777.",2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,1533047703118,ix-icicib,496.0,0,0.0,0,0.0,0,0.0,0,0.0,1.0,inr 200.00,0


In [78]:
Final = Final[['id','gUserId','deviceId','createdDate','modifiedDate','messageId','message','smsDate','sender_name','Category','Debited Amount','Credited Amount','Credit_Card_Holder','ongoing_loan','EMI_amount','User_is_defaulter','Overdue_amount','paytm_user','Paytm_debit_trans_amount','Paytm_credit_trans_amount']]

In [80]:
Final.columns = ['Id','GUserId','DeviceId','CreatedDate','ModifiedDate','MessageId','SMS','SmsDate','Sender_name','Category','Debited_Amount','Credited_Amount','Credit_Card_Holder','Ongoing_loan','EMI_of_loan','User_is_defaulter','Pending_EMI','Paytm_user','Paytm_debit_amount','Paytm_credit_amount'] 

In [81]:
Final

,Id,GUserId,DeviceId,CreatedDate,ModifiedDate,MessageId,SMS,SmsDate,Sender_name,Category,Debited_Amount,Credited_Amount,Credit_Card_Holder,Ongoing_loan,EMI_of_loan,User_is_defaulter,Pending_EMI,Paytm_user,Paytm_debit_amount,Paytm_credit_amount
0,110365150,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,314.0,rs.12600 is debited to a/c ...1545 on 09-08-18 12:56:26 (avlbl bal rs.212.6). tollfree 1800223344 18001024455 (24x7),1533799814965,axbobtxn,1.0,rs.12600,0,0.0,0.0,0,0.0,0,0.0,0,0
1,110365151,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,315.0,your vpa 7041699604@ybl linked to your a/c no. xxxxxx1545 is debited for rs. 200.0 and credited to vpa fxm@ybl upi ref no. 822160553315.,1533807964427,axbobsms,1.0,rs. 200.0,0,0.0,0.0,0,0.0,0,0.0,0,0
2,110365155,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,319.0,rs.200 is debited to a/c ...1545 on 09-08-18 15:16:01 (avlbl bal rs.12.6). tollfree 1800223344 18001024455 (24x7),1533808303294,axbobtxn,1.0,rs.200,0,0.0,0.0,0,0.0,0,0.0,0,0
3,110365158,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,326.0,your ac xxxxx915232 debited inr 13.44 on 09/08/18 -mab sb debit . avl bal inr 186.56.plz download buddy,1533846772103,idcbssbi,1.0,inr 13.44,0,0.0,0.0,0,0.0,0,0.0,0,0
4,110365161,076b042a-5af4-47d3-a07d-36e20608a927,bce1f870eae211cf,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,330.0,"dear sbi upi user, your account is debited inr 86.0 on date 2018-08-10 10:13:44 am by upi ref no 822240622992.download yono @ www.yonosbi.com",1533876233446,adsbiupi,1.0,inr 86.0,0,0.0,0.0,0,0.0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116795,110705978,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,157.0,you have initiated a txn of inr 696.00 at paytm on icici bank card no.0803. otp is 693391. dont share with anyone. bank never calls to verify otp.,1530627485461,tm-icicib,0.0,0,0,0.0,0.0,0,0.0,0,1.0,inr 696.00,0
116796,110705979,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,158.0,"dear customer,txn of inr 696.00 done via debit card of acct xx4312 on 03-jul-18.info: iin*paytm .avbl bal:inr 5,409.26.for dispute,call 04033667777.",1530627493139,ix-icicib,0.0,0,0,0.0,0.0,0,0.0,0,1.0,inr 696.00,0
116797,110706040,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,382.0,delight with a flight! gift a moment of a lifetime with indigo gift vouchers. get 5% cashback upto inr100 on transactions through paytm https://bit.ly/2cicego,1532495930947,ax-indoff,0.0,0,0,0.0,0.0,0,0.0,0,1.0,0,0
116798,110706067,6814c6df-a64e-423d-81a0-bb2c589b77d2,c617cb034ecccac8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,496.0,"dear customer,txn of inr 200.00 done via debit card of acct xx4312 on 31-jul-18.info: iin*paytm .avbl bal:inr 25,621.46.for dispute,call 04033667777.",1533047703118,ix-icicib,0.0,0,0,0.0,0.0,0,0.0,0,1.0,inr 200.00,0


In [82]:
Final = Final.apply(lambda x: x.astype(str).str.upper())

In [83]:
Final

,Id,GUserId,DeviceId,CreatedDate,ModifiedDate,MessageId,SMS,SmsDate,Sender_name,Category,Debited_Amount,Credited_Amount,Credit_Card_Holder,Ongoing_loan,EMI_of_loan,User_is_defaulter,Pending_EMI,Paytm_user,Paytm_debit_amount,Paytm_credit_amount
0,110365150,076B042A-5AF4-47D3-A07D-36E20608A927,BCE1F870EAE211CF,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,314.0,RS.12600 IS DEBITED TO A/C ...1545 ON 09-08-18 12:56:26 (AVLBL BAL RS.212.6). TOLLFREE 1800223344 18001024455 (24X7),1533799814965,AXBOBTXN,1.0,RS.12600,0,0.0,0.0,0,0.0,0,0.0,0,0
1,110365151,076B042A-5AF4-47D3-A07D-36E20608A927,BCE1F870EAE211CF,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,315.0,YOUR VPA 7041699604@YBL LINKED TO YOUR A/C NO. XXXXXX1545 IS DEBITED FOR RS. 200.0 AND CREDITED TO VPA FXM@YBL UPI REF NO. 822160553315.,1533807964427,AXBOBSMS,1.0,RS. 200.0,0,0.0,0.0,0,0.0,0,0.0,0,0
2,110365155,076B042A-5AF4-47D3-A07D-36E20608A927,BCE1F870EAE211CF,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,319.0,RS.200 IS DEBITED TO A/C ...1545 ON 09-08-18 15:16:01 (AVLBL BAL RS.12.6). TOLLFREE 1800223344 18001024455 (24X7),1533808303294,AXBOBTXN,1.0,RS.200,0,0.0,0.0,0,0.0,0,0.0,0,0
3,110365158,076B042A-5AF4-47D3-A07D-36E20608A927,BCE1F870EAE211CF,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,326.0,YOUR AC XXXXX915232 DEBITED INR 13.44 ON 09/08/18 -MAB SB DEBIT . AVL BAL INR 186.56.PLZ DOWNLOAD BUDDY,1533846772103,IDCBSSBI,1.0,INR 13.44,0,0.0,0.0,0,0.0,0,0.0,0,0
4,110365161,076B042A-5AF4-47D3-A07D-36E20608A927,BCE1F870EAE211CF,2018-08-25 07:42:14.313000,2018-08-25 07:42:14.313000,330.0,"DEAR SBI UPI USER, YOUR ACCOUNT IS DEBITED INR 86.0 ON DATE 2018-08-10 10:13:44 AM BY UPI REF NO 822240622992.DOWNLOAD YONO @ WWW.YONOSBI.COM",1533876233446,ADSBIUPI,1.0,INR 86.0,0,0.0,0.0,0,0.0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116795,110705978,6814C6DF-A64E-423D-81A0-BB2C589B77D2,C617CB034ECCCAC8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,157.0,YOU HAVE INITIATED A TXN OF INR 696.00 AT PAYTM ON ICICI BANK CARD NO.0803. OTP IS 693391. DONT SHARE WITH ANYONE. BANK NEVER CALLS TO VERIFY OTP.,1530627485461,TM-ICICIB,0.0,0,0,0.0,0.0,0,0.0,0,1.0,INR 696.00,0
116796,110705979,6814C6DF-A64E-423D-81A0-BB2C589B77D2,C617CB034ECCCAC8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,158.0,"DEAR CUSTOMER,TXN OF INR 696.00 DONE VIA DEBIT CARD OF ACCT XX4312 ON 03-JUL-18.INFO: IIN*PAYTM .AVBL BAL:INR 5,409.26.FOR DISPUTE,CALL 04033667777.",1530627493139,IX-ICICIB,0.0,0,0,0.0,0.0,0,0.0,0,1.0,INR 696.00,0
116797,110706040,6814C6DF-A64E-423D-81A0-BB2C589B77D2,C617CB034ECCCAC8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,382.0,DELIGHT WITH A FLIGHT! GIFT A MOMENT OF A LIFETIME WITH INDIGO GIFT VOUCHERS. GET 5% CASHBACK UPTO INR100 ON TRANSACTIONS THROUGH PAYTM HTTPS://BIT.LY/2CICEGO,1532495930947,AX-INDOFF,0.0,0,0,0.0,0.0,0,0.0,0,1.0,0,0
116798,110706067,6814C6DF-A64E-423D-81A0-BB2C589B77D2,C617CB034ECCCAC8,2018-08-25 12:48:53.810000,2018-08-25 12:48:53.810000,496.0,"DEAR CUSTOMER,TXN OF INR 200.00 DONE VIA DEBIT CARD OF ACCT XX4312 ON 31-JUL-18.INFO: IIN*PAYTM .AVBL BAL:INR 25,621.46.FOR DISPUTE,CALL 04033667777.",1533047703118,IX-ICICIB,0.0,0,0,0.0,0.0,0,0.0,0,1.0,INR 200.00,0


In [87]:
Final.to_csv(r'C:\Users\nitishkumar\Desktop\P\Data-\Fullerton Msg\SMS_DATA\Raw sms\Transdatapython27thDEC.csv',index = False)